In [5]:
import spacy
import re
import pandas as pd
import nltk
from nltk.stem.snowball import SnowballStemmer
import string

In [10]:
nlp = spacy.load("fr_core_news_sm")
stopwords = list(nlp.Defaults.stop_words)
punctuation = list(string.punctuation)
s_stemmer = SnowballStemmer("french")

In [13]:
df = pd.read_csv('database.csv',parse_dates=['publishedAt','extractedAt'])
df.head()

,channelId,videoId,author,publishedAt,comment,extractedAt
0,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@SEBFRIT,2025-06-02 19:21:45+00:00,Merci à Odoo le sponsor cette vidéo ! Pour déc...,2025-07-01 11:46:10
1,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@guishoo4700,2025-06-30 18:45:12+00:00,Fait une vidéo sur Tyler the creator stp,2025-07-01 11:46:10
2,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@Remi-B-Goode,2025-06-30 16:35:49+00:00,Lol le disclaimer😂,2025-07-01 11:46:10
3,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@Guaph,2025-06-29 20:17:58+00:00,"où peut-on voir la série ""Earn and Alfred"" ???",2025-07-01 11:46:10
4,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@rockman156,2025-06-29 19:05:12+00:00,C'est pas un re upload ? Impression de déjà vu...,2025-07-01 11:46:10


In [11]:
def reduire_repetitions(mot):
    # nettoyage des répétitions de lettres
    # On remplace les répétitions de lettres par une seule occurrence
    # Ex: "loooove" devient "love"
    return re.sub(r'(.)\1{2,}', r'\1', mot)

def preprocessing(text, join=True, methode='lemma', extra_stopwords : list = None, extrta_pattern : re.Pattern=None):
    if extra_stopwords is not None:
        nondesiredtokens = set(list(stopwords)+extra_stopwords)
    else:
        nondesiredtokens = set(stopwords)
    pattern = re.compile(r"(http://\S+|@\S+|.*\d.*|.*\#.*)") 

    # Tokenisation et nettoyage en une seule passe
    if methode == 'lemma':
        tokens = [pattern.sub("", token.lemma_.lower())  for token in nlp(text) if not token.is_stop and token.text.lower() not in nondesiredtokens and len(token) > 3]
    elif methode == 'stem':
        tokens = [pattern.sub("", s_stemmer.stem(token.text.lower()))  for token in nlp(text) if not token.is_stop and token.text.lower() not in nondesiredtokens and len(token) > 3]
    
    # Filtrer les tokens vides après suppression des liens/mentions
    cleaned_tokens = [re.sub(r'\-{1,}|\\{1,}|\/{1,}|\_{1,}|~{1,}|\*{1,}|\?{1,}|\={1,}|\:{1,}|\){1,}|\({1,}', '', reduire_repetitions(token)) for token in tokens if len(token)>3]
    
    return " ".join(cleaned_tokens) if join else cleaned_tokens

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1731 entries, 0 to 1730
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   channelId    1731 non-null   object             
 1   videoId      1731 non-null   object             
 2   author       1727 non-null   object             
 3   publishedAt  1731 non-null   datetime64[ns, UTC]
 4   comment      1730 non-null   object             
 5   extractedAt  1731 non-null   datetime64[ns]     
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), object(4)
memory usage: 81.3+ KB


In [15]:
# pattern = re.compile(r"(http://\S+|@\S+|.*\d.*|.*\#.*)") 
df['comment_clean_lem'] = df['comment'].astype(str).apply(lambda x: preprocessing(x, join=False))
df.head(50)

,channelId,videoId,author,publishedAt,comment,extractedAt,comment_clean_lem
0,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@SEBFRIT,2025-06-02 19:21:45+00:00,Merci à Odoo le sponsor cette vidéo ! Pour déc...,2025-07-01 11:46:10,"[odoo, sponsor, vidéo, découvrir, odoo, clique..."
1,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@guishoo4700,2025-06-30 18:45:12+00:00,Fait une vidéo sur Tyler the creator stp,2025-07-01 11:46:10,"[vidéo, tyler, creator]"
2,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@Remi-B-Goode,2025-06-30 16:35:49+00:00,Lol le disclaimer😂,2025-07-01 11:46:10,[disclaimer]
3,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@Guaph,2025-06-29 20:17:58+00:00,"où peut-on voir la série ""Earn and Alfred"" ???",2025-07-01 11:46:10,"[voir, série, earn, alfred]"
4,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@rockman156,2025-06-29 19:05:12+00:00,C'est pas un re upload ? Impression de déjà vu...,2025-07-01 11:46:10,"[upload, impression, critique, foireux, impres..."
5,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@r_bin,2025-06-29 16:30:34+00:00,fais une vidéo sur tyler the creator stpppppppp,2025-07-01 11:46:10,"[vidéo, tyler, creator, stp]"
6,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@collet_1311,2025-06-29 05:15:45+00:00,C est pas SEB c est un robot 😅😅😅,2025-07-01 11:46:10,[robot]
7,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@vladimiro.6067,2025-06-28 11:45:19+00:00,"Super vidéo comme d’hab, mais je trouve comme ...",2025-07-01 11:46:10,"[super, vidéo, trouver, trop, individualiste, ..."
8,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@CopainCodex,2025-06-28 02:33:23+00:00,la coiffure et le rasage ne me dérange pas le ...,2025-07-01 11:46:10,"[coiffure, rasage, déranger, monde, contre, gr..."
9,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@clem2023,2025-06-27 17:28:13+00:00,Super vidéo Seb merci vraiment mais par contre...,2025-07-01 11:46:10,"[super, vidéo, vraiment, contre, faillir, tape..."


# Écriture du fichier

In [16]:
df.to_csv('database_clean.csv', index=False)